In [136]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
from datetime import datetime
import base as bs

import os as os
import csv

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [137]:
def get_url(art):
    try:
        url = art.find_element(By.XPATH, '//h2[@class="c_t"]//a').get_attribute('href')
    except:
        url = None
    else:
        return url


In [138]:
def get_titulo(art):
    try:
        titulo = art.find_element(By.XPATH, '//h2[@class="c_t"]//a').text
    except:
        titulo = None
    else:
        return titulo

In [139]:
def get_autor(art):
    try:
        autor = art.find_element(By.XPATH, '//div[@class="c_a"]//a').text
    except:
        autor = None
    else:
        return autor

In [140]:
def get_fecha(art):
    try:
        fecha = art.find_element(By.XPATH, '//div[@class="c_f"]').text.split("\n")[0]
        fecha_pub = datetime.strptime(fecha, '%d/%m/%Y')
    except:
        fecha_pub = None
    return fecha_pub

In [141]:
def get_resumen(art):
    try:
        resumen = art.find_element(By.XPATH, './/p').text
    except:
        resumen = None
    return resumen

In [142]:
def get_tema(art):
    try:
        tema = art.find_element(By.XPATH, '//header[contains(@class, "c_h")]/a').text
    except:
        tema = None
    return tema

In [143]:
def get_imag(art):
    try:
        imagen = art.find_element(By.XPATH, './/img').get_attribute('src')
    except:
        imagen = None
    return imagen

In [144]:
def get_ciudad(driver):
    try:
        ciudad = driver.find_element(By.XPATH, './/div[@class="a_md_f"]').text.split(' ')[0]
    except:
        ciudad = None
    return ciudad

In [145]:
def get_contenido(driver):
    try:
        parrafos = driver.find_elements(By.XPATH, '//div[@class="a_c clearfix"]/p')
        contenido = '\n\n'.join(list(map(lambda x: x.text, parrafos)))
    except:
        contenido = None
    return contenido

In [146]:
#Empresa con la cual vamos a extraer los articulos
empresa = input("Digite la empresa a extraer: ").lower()
revista = "elPais"

In [147]:
# crear driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://elpais.com/buscador/?q={empresa}')
time.sleep(2)

# Si en 5 segundo no ha encontrado el elemento que se le pasa se detiene
# Luego acepta los cookies
WebDriverWait(driver, 5)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      "button.didomi-components-button didomi-button didomi-dismiss-button didomi-components-button--color didomi-button-highlight highlight-button".replace(" ", "."))))\
                                          .click()
time.sleep(2)
driver.execute_script('window.scrollBy(0, document.body.scrollHeight)')                                  

In [148]:
# "c c-d _g _g-md c-m-l c--m-n" Contiene tema
articulos_Imag = driver.find_elements(By.XPATH, './/article[contains(@class, "c c-d _g _g-md c-m-l c--m")]')
articulos_NoImag = driver.find_elements(By.XPATH, './/article[contains(@class, "c c-d _g _g-md c-m-l c--m-n")]')
articulos = articulos_Imag + articulos_NoImag


In [149]:
titulares = []
for art in articulos:
    # if not(bs.existedb(url, revista)):
    titulo = get_titulo(art)
    autor = get_autor(art)
    fecha = get_fecha(art)
    resumen = get_resumen(art)
    tema = get_tema(art)
    if art in articulos_Imag:
        imagen = get_imag(art)
    else:
        imagen = None
    titulares.append({'Fecha Extraccion':datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'Titulo': titulo,
                    'Fecha Publicacion':fecha,
                    'Tema': tema,
                    'Resumen': resumen,
                    'URL': url,
                    'Imagen': imagen,
                    'Empresa': empresa,
                    'Autor': autor
                    })

In [151]:
# Buscar el contenido de cada artículo y los almacena en la lista de titulares
for tit in titulares:
    driver.get(tit['URL'])
    
    tit['Contenido'] = get_contenido(driver)
    # tit['Ciudada de publicación'] = get_ciudad(driver)

In [ ]:
len(titulares)

In [152]:
columnas = ['Fecha Extraccion', 'Titulo', 'Fecha Publicacion', 'Tema',
            'Resumen', 'URL', 'Imagen', 'Empresa', 'Autor', 'Contenido']

In [129]:
# Para AGREGAR titulares a un archivo.
with open(f'../data/raw/{revista}.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [153]:
#Para crear un NUEVO archivo con los titulares
pd.json_normalize(titulares).to_csv(f'../data/raw/{revista}.csv',index=False, encoding='latin-1', errors='ignore', columns=columnas)

In [131]:
# Demora inteligente
# delay = 10
# try:
#     url = WebDriverWait(driver, delay).until(EC.presence_of_element_located(By.XPATH, ".//div"))
# except:
#     print("La página tardó demasiado en cargar")